In [1]:
from IPython.display import HTML
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

%matplotlib inline

In [3]:
df = pd.read_csv('../output/measure_doac_rx_mechanical_valve.csv')
df = df.dropna(axis=0)
df = df[df['doac']>5]


selected_stps = []
for stp in df['stp'].unique():
    subset = df[df['stp']==stp]
    if subset['doac'].min() >5:
        selected_stps.append(stp)


stps = selected_stps[0:20]
df[df['stp'].isin(stps)].to_csv('../output/check.csv')
num_per_thousand = df['doac']/(df['population']/1000)
df['rate'] = num_per_thousand

In [6]:
def add_percentiles(df, period_column=None, column=None, show_outer_percentiles=True):
    """For each period in `period_column`, compute percentiles across that
    range.
    Adds `percentile` column.
    """
    deciles = np.arange(0.1, 1, 0.1)
    bottom_percentiles = np.arange(0.01, 0.1, 0.01)
    top_percentiles = np.arange(0.91, 1, 0.01)
    if show_outer_percentiles:
        quantiles = np.concatenate((deciles, bottom_percentiles, top_percentiles))
    else:
        quantiles = deciles
    df = df.groupby(period_column)[column].quantile(quantiles).reset_index()
    df = df.rename(index=str, columns={"level_1": "percentile"})
    # create integer range of percentiles
    df["percentile"] = df["percentile"].apply(lambda x: int(x * 100))
    return df


def interactive_deciles_chart(
    df,
    practices,
    period_column=None,
    column=None,
    title="",
    ylabel="", 
    ):
    """period_column must be dates / datetimes
    """
    
    original_df = df
    
    df = add_percentiles(
        df,
        period_column=period_column,
        column=column,
        show_outer_percentiles=False,
    )
 
    fig = go.Figure()

    linestyles = {
        "decile": {"color": "blue", "dash": "dash"},
        "median": {"color": "blue", "dash": "solid"},
        "percentile": {"color": "blue", "dash": "dash"},
    }
    

    
    for percentile in np.unique(df['percentile']):
        df_subset = df[df['percentile'] == percentile]
        if percentile == 50:
            fig.add_trace(go.Scatter(x=df_subset[period_column], y=df_subset[column], line={
                          "color": "blue", "dash": "solid", "width": 1.2}, name="median", visible=True))
            
        else:
            fig.add_trace(go.Scatter(x=df_subset[period_column], y=df_subset[column], line={
                          "color": "blue", "dash": "dash", "width": 1}, name=f"decile {int(percentile/10)}", visible=True))

    
    for i, stp in enumerate(stps):
        df_subset = original_df[original_df['stp']==stp]
        fig.add_trace(go.Scatter(x=df_subset[period_column], y=df_subset[column], line={
                          "color": "red", "dash": "solid", "width": 1}, name=f"STP {i}", visible=False))
   
    
    buttons_list=[]
    for i in range(len(stps)):
        visible = [False]*len(stps)
        visible[i] = True
        
        buttons_list.append(dict(args = [{'visible': [True]*9 + visible}],label=str(stps[i]),method="update"))
        

    buttons_list.insert(0, dict(args = [{'visible':  [True]*9 + [False]*len(stps) }],label="All",method="restyle"))
    
    
                
     # Set title
    fig.update_layout(
        title_text=title,
        hovermode='x',
        title_x=0.5,


    )

    fig.update_yaxes(title=ylabel)
    fig.update_xaxes(title="Date")

    # Add range slider
    fig.update_layout(
        xaxis=go.layout.XAxis(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1m",
                         step="month",
                         stepmode="backward"),
                    dict(count=6,
                         label="6m",
                         step="month",
                         stepmode="backward"),

                    dict(count=1,
                         label="1y",
                         step="year",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
        
    
    )
    
    fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons_list,
            direction="down",
            pad={"r": 10, "t": 10},
           
            x=1.0,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

    fig.update_layout(showlegend=False)
    fig.show()


## DOAC Use in Patients with Mechanical Heart Valves

### Total Number

In [17]:
total_df = df.groupby(by='date')[['doac', 'population']].sum().reset_index()

def plot_measures(df, title,column_to_plot, category=False, y_label='Rate per 1000', interactive=True):

    if interactive:

        fig = go.Figure()

        if category:
            for unique_category in df[category].unique():

                df_subset = df[df[category] == unique_category]
                fig.add_trace(go.Scatter(
                    x=df_subset['date'], y=df_subset[column_to_plot], name=unique_category))

        else:
            fig.add_trace(go.Scatter(
                x=df['date'], y=df[column_to_plot]))

        # Set title
        fig.update_layout(
            title_text=title,
            hovermode='x',
            title_x=0.5,


        )

        fig.update_yaxes(title=y_label)
        fig.update_xaxes(title="Date")

        # Add range slider
        fig.update_layout(
            xaxis=go.layout.XAxis(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                            label="1m",
                            step="month",
                            stepmode="backward"),
                        dict(count=6,
                            label="6m",
                            step="month",
                            stepmode="backward"),

                        dict(count=1,
                            label="1y",
                            step="year",
                            stepmode="backward"),
                        dict(step="all")
                    ])
                ),
                rangeslider=dict(
                    visible=True
                ),
                type="date"
            )
        )

        fig.show()

    else:

        if category:
            for unique_category in df[category].unique():

                df_subset = df[df[category] == unique_category]

                plt.plot(df_subset['date'], df_subset[column_to_plot], marker='o')
        else:
            plt.plot(df['date'], df[column_to_plot], marker='o')

        plt.ylabel(y_label)
        plt.xlabel('Date')
        plt.xticks(rotation='vertical')
        plt.title(title)

        if category:
            plt.legend(df[category].unique(), bbox_to_anchor=(
                1.04, 1), loc="upper left")

        else:
            pass

        plt.show()
        plt.clf()

In [19]:
plot_measures(total_df, title='', column_to_plot='doac', category=False, y_label='Total', interactive=True)


### Rate per 1000 by STP

In [20]:
interactive_deciles_chart(df,selected_stps,period_column='date',column='rate',title="",ylabel="Rate per 1000")